# LangChain 核心模块 Agent - OpenAI Function

In [1]:
import os
import configparser

conf = configparser.ConfigParser()
current_directory = os.path.dirname(os.path.realpath('__file__'))
config_file_path = os.path.join(current_directory, '..', '..', '..', 'config.ini')
conf.read(config_file_path)
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
os.environ["OPENAI_API_KEY"] = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["SERPAPI_API_KEY"] = conf.get("SerpApi", "serp_apikey")  # 在config.ini中配置自己的APIkey
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [2]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [3]:

from langchain.agents import tool


@tool
def get_word_length(words: str) -> int:
    """Return the length of a word."""
    return len(words)


tools = [get_word_length]


In [4]:
from langchain.agents import OpenAIFunctionsAgent
from langchain.schema import SystemMessage

system_message = SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)


In [5]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [6]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.run("how many letters in the word AgentExecutor?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'words': 'AgentExecutor'}`


13There are 13 letters in the word "AgentExecutor".

> Finished chain.


'There are 13 letters in the word "AgentExecutor".'

In [7]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [9]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
agent_executor.run("how many letters in the word AgentExecutor?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'words': 'AgentExecutor'}`


13There are 13 letters in the word "AgentExecutor".

> Finished chain.


'There are 13 letters in the word "AgentExecutor".'

In [10]:
agent_executor.run("is that a real word?")




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'words': 'AgentExecutor'}`


13

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-x42xnbIQNTmMDq6adIeja9TZ on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


No, "AgentExecutor" is not a real word. It is a combination of the words "Agent" and "Executor".

> Finished chain.


'No, "AgentExecutor" is not a real word. It is a combination of the words "Agent" and "Executor".'